In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir('/content/drive/MyDrive/Colab Notebooks/Hackerton')

In [ ]:
!pip install --upgrade wandb -qq

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 146 kB 89.6 MB/s 
     |████████████████████████████████| 181 kB 104.8 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 


In [ ]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os, sys
import numpy as np
from pathlib import Path
import pandas as pd
from pprint import pprint
import easydict
import wandb
import torch

from lane_change_risk_detection.dataset import DataSet
from lane_change_risk_detection.models import LSTM_Classifier, CNN_LSTM_Classifier, CNN_Classifier, ResNet50_LSTM_Classifier 
from lane_change_risk_detection.train import Trainer

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

PROJECT_NAME = "socar_hackerton"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
args_parsed = easydict.EasyDict({
  'input_path': '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/masked_images',
	'pkl_path':'/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_dataset.pkl',
	'load_pkl': True,
	'save_pkl_path':'/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_dataset.pkl',
	'save_pkl': True,
  'model_name': 'cnn_lstm',
  'n_folds': 1,
  'train_ratio': 0.9,
  'downsample': False,
  'seed': 0,
  'test_step': 10,
  'device': 'cuda',
  'epochs': 100,
  'batch_size': 32,
  'bnorm': False,
  'dropout': 0.1,
  'learning_rate': 3e-5,
  'weight_decay': 5e-4 

   })
class Config:

  def __init__(self, args):
    self.args_parsed = args
    self.wandb = wandb.init(project=PROJECT_NAME)
    self.wandb_config = self.wandb.config

    for arg_name in vars(self.args_parsed):
      self.__dict__[arg_name] = getattr(self.args_parsed, arg_name)
      self.wandb_config[arg_name] = getattr(self.args_parsed, arg_name)
    self.wandb_base_dir = Path('/content/drive/MyDrive/Colab Notebooks/Hackerton/data/masked_images').resolve

def load_dataset(raw_image_path, risk_path, config=None):
  dataset = DataSet()
  dataset.read_video(raw_image_path, option='fixed frame amount', number_of_frames = 50, scaling = 'scale', scale_x = 0.1, scale_y = 0.1)
  dataset.read_risk_data(risk_path)
  dataset.convert_risk_to_one_hot(risk_threshold = 0.05)
  if config != None and config.save_pkl:
    parent_path = '/'.join(config.save_pkl_path.split('/')[:-1]) + '/'
    fname = config.save_pkl_path.split('/')[-1]
    dataset.save(save_dir = parent_path, filename=fname)
    print('Saved pickled dataset')
  return dataset


def load_pickle(pkl_path):
	dataset = DataSet().loader(str(pkl_path))
	return dataset



def reshape_dataset(dataset):
	'''
		input -> (batch, frames, height, width, channels)
		output -> (batch, frames, channels, height, width)
	'''
	return np.swapaxes(np.swapaxes(dataset, -1, -3), -1, -2)

def train_model(dataset, config):
	pth_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'
	dataset.video = reshape_dataset(dataset.video)
	video_sequences = dataset.video
	labels = dataset.risk_one_hot
	print(len(labels))
	clip_names = np.array(['default_all']*len(video_sequences))
	if hasattr(dataset, 'foldernames'): 
		clip_names = np.concatenate((clip_names, dataset.foldernames), axis=0)

	if config.model_name == 'gru':
		model = LSTM_Classifier(video_sequences.shape, 'gru', config)
	elif config.model_name == 'lstm':
		model = LSTM_Classifier(video_sequences.shape, 'lstm', config)
	elif config.model_name == 'cnn':
		model = CNN_Classifier(video_sequences.shape, config)
	elif config.model_name == 'cnn_lstm':
		model = CNN_LSTM_Classifier(video_sequences.shape, config)
	elif config.model_name == 'resnet': 
		model = ResNet50_LSTM_Classifier(video_sequences.shape, config)
	
	trainer = Trainer(config)
	trainer.init_dataset(video_sequences, labels, clip_names)
	trainer.build_model(model)
 
	if config.n_folds > 1:
		trainer.train_n_fold_cross_val()
	else:
		trainer.train_model()
	torch.save(model.state_dict(), pth_path)
 

config = Config(args_parsed)
image_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/input'
risk = '/content/drive/MyDrive/Colab Notebooks/Hackerton/test/LCTable.csv'

#if config.load_pkl:
#	dataset = load_pickle(Path(config.pkl_path).resolve())
#else:
#	dataset = load_dataset(image_path, risk, config=config)

	
# train model
#model = train_model(dataset, config)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
im_path = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/test_input/'

dataset = DataSet()
dataset.read_video(im_path, option='fixed frame amount', number_of_frames = 50, scaling = 'scale', scale_x = 0.1, scale_y = 0.1)
dataset.video = reshape_dataset(dataset.video)
video_sequences = dataset.video

#print(video_sequences.shape)
#cnn_lstm = CNN_LSTM_Classifier(video_sequences.shape, config)
#cnn_lstm.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'))
model = CNN_LSTM_Classifier(video_sequences.shape, config)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/Hackerton/save_train_data/save_parameter.pth'))

100%|██████████| 2/2 [00:10<00:00,  5.18s/it]


<All keys matched successfully>

In [ ]:
print(video_sequences.shape)

(5, 50, 3, 86, 154)


In [ ]:
type(video_sequences)
#model.forward(video_sequences)

numpy.ndarray

In [ ]:
video_sequences.shape

(1, 50, 3, 86, 154)

In [ ]:
torch_video_sequences = torch.Tensor(video_sequences)
type(torch_video_sequences)

torch.Tensor

In [ ]:
with torch.no_grad():
    model.eval()
    inputs = torch_video_sequences
    outputs = model(inputs)
    print(outputs)

tensor([[0.8118, 0.1882],
        [0.8630, 0.1370]])


In [ ]:
from PIL import Image
root = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/input/1/1.jpg'
img1 = Image.open(root)
print(img1.size)

(1280, 720)


In [ ]:
root2 = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/test_input/1/1.jpg'
img2 = Image.open(root2)
print(img2.size)

(1280, 720)


In [ ]:
root3 = '/content/drive/MyDrive/Colab Notebooks/Hackerton/data/test_input/2/1.jpg'
img3 = Image.open(root3)
print(img3.size)

(1024, 450)
